In [ ]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import serial
import tdwf
import time
import os

#-[Configurazione seriale]-----------------------------------------------------
ser = serial.Serial()
ser.port = "COM7"
ser.baudrate = 115200
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n'
        ser.write(message.encode('utf-8'))
    if flagR:
        res = ser.readline().decode('utf-8')
        return res


for i in range(11):
    response = ser.readline().decode('utf-8')
    print(response[:-1])


#-[Configurazione AD2]-------------------------------------------------------
ad2 = tdwf.AD2(iconfig=2)

ad2.vdd = +5
ad2.power(True)

#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.config(ampl = 2.5, offs= 2.5, func=tdwf.funcSquare, freq = 10, duty= 1)
wgen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=2e5
scope.npt=16384
scope.ch1.rng = 10
scope.ch2.rng = 10
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1, delay = 0.04)


#-[Saving pipeline]-----------------------------------------------------
Directory = f"distances_{time.strftime('%Y%m%d_%H%M%S')}" # directory name based on the current date and time
parent_dir = "../../Data"
path = os.path.join(parent_dir, Directory) # creating the directory
os.mkdir(path)
print(f"Data will be saved in {path}\n")
log_file = os.path.join(path, f"log.txt")   #creating the log file



#-[Plotting setup]-----------------------------------------------------


# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq
    if event.key == 'x':  # => export su file
        filename = input("Esporta dati su file: ")
        data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))
        if scope.npt > 8192:
            info =  f"Acquisizione Analog Discovery 2 - Lunga durata\ntime\tch1\tch2"
        else:
            info =  f"Acquisizione Analog Discovery 2\nTimestamp {scope.time.t0}\ntime\tch1\tch2"
        np.savetxt(filename, data, delimiter='\t', header=info)
    if event.key == ' ':  # => run/pausa misura
        flag_acq = not flag_acq
    if event.key == 'escape':  # => esci dalla misura
        flag_run = False


fig, ax = plt.subplots(figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('key_press_event', on_key)
flag_run = True
flag_acq = True
flag_first = True

hp1, = plt.plot(scope.time.vals, scope.ch1.vals, "-", label="Trigger", color="tab:orange")
hp2, = plt.plot(scope.time.vals, scope.ch2.vals, "-", label="Echo", color="tab:blue")
plt.legend()
plt.grid(True)
plt.xlabel("Time [s]", fontsize=15)
plt.ylabel("Signal [V]", fontsize=15)
plt.title("User interaction: x|space|escape")
plt.ylim(-1,6)
plt.tight_layout()
plt.show(block = False)




# read data from HC-SRF05, move to next position, and read again

rangeX = [0, 70000]
posX = 0
maxIterations = 10
counter = 0

while posX < 70000:

    print(f"Distance Measurement - X = {posX:.0f}")

    if not flag_run:
        break
    if counter > maxIterations:
        break


    resp = query('TX ' + str(posX), flagW=True, flagR=True)
    print(resp[:-1])


    data = query('x?')




    while int(data.split('=')[1]) != posX:
        data = query('x?')


    time.sleep(0.5)

    scope.sample()
    hp1.set_ydata(scope.ch1.vals)
    hp2.set_ydata(scope.ch2.vals)
    fig.canvas.draw()
    fig.canvas.flush_events()




    info = f"Distance Measurement - X = {posX:.0f}\n"

    data_to_save = np.c_[scope.time.vals, scope.ch1.vals, scope.ch2.vals]
    info = info + f"time [s]\tch1 [V]\tch2 [V]\n"


    filename = f"distanceMeasurement_{posX:.0f}"

    np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")



    counter += 1

    posX += int( (70000 - posX) /2 )



Digilent WaveForms SDK versione 3.22.2











Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #2
Data will be saved in ../../Data\distances_20250508_102023

Distance Measurement - X = 0
TX=0
Distance Measurement - X = 35000
TX=35000
Distance Measurement - X = 52500
TX=52500
Distance Measurement - X = 61250
TX=61250
Distance Measurement - X = 65625
TX=65625
Distance Measurement - X = 67812
TX=67812
Distance Measurement - X = 68906
TX=68906
Distance Measurement - X = 69453
TX=69453
Distance Measurement - X = 69726
TX=69726
Distance Measurement - X = 69863
TX=69863
Distance Measurement - X = 69931
TX=69931
Distance Measurement - X = 69965


In [1]:
data = query(input('>> '), flagR=True, flagW=True)
      
print(data[:-1])

NameError: name 'query' is not defined

In [6]:
    print( data.split('=')[1] )

    print(posX)

    print(type(data.split('=')[1]))
    print(type(posX))

0

0
<class 'str'>
<class 'int'>


In [3]:
ad2.close()

Dispositivo disconnesso.


In [8]:
ser.close()